# OLAP Operationen
OLAP steht für **O**n**l**ine **A**nalytical **P**rocessing und bezeichnet eine Methode, Daten aus verschiedenen Blickwinkeln zu erfassen und zu analysieren. Als Basis dafür dient ein Ergebniswürfel.

## Inhaltsverzeichnis
- [Vorbereitung](#Vorbereitung)
- [Pivotierung](#Pivotierung)
- [Roll-Up und Drill-Down](#Roll-Up-und-Drill-Down)
- [Drill-Across](#Drill-Across)
- [Slice and Dice](#Slice-and-Dice)

## Vorbereitung
Zunächst wird Pandas geladen und der Ergebniswürfel des vorigen Abschnitts erneut erzeugt.

Eventuell ist Ihnen bereits im vorangegangenen Abschnitt aufgefallen, dass beispielsweise für `Act`, `MiS`, `1990` keine Daten vorliegen und diese in der Aggregation entsprechend fehlen. Um den Würfel farbig markieren zu können ist jedoch ein vollständiger Index notwendig, sodass die Erstellung des Ergebniswürfels in der nachfolgenden Zelle geringfügig verändert wurde.

In [ ]:
import pandas as pd
from cube import draw

df = pd.read_csv('vgsales.csv', index_col='rank')

cube = df.groupby(['decade', 'genre', 'platform'])['sales'].mean()
cube = cube.unstack(0).stack(0, dropna=False).fillna(0)
cube = cube.round(2)

cube.head(15)

In [ ]:
draw(cube)

## Pivotierung
Die **Pivotierung** oder auch Rotation beschreibt ein Vertauschen der Dimensionen, um einen anderen Blickwinkel zu ermöglichen. In Pandas steht die Funktion `swaplevel` bereit, um zwei Indizes zu vertauschen. Die Funktion verändert nicht die Reihenfolge der Elemente.

In [ ]:
cube.swaplevel(i=0, j=2).head(15)

In [ ]:
draw(
    cube,
    cube.swaplevel(i=0, j=2)
)

## Roll-Up und Drill-Down
**Roll-Up** beschreibt eine Operation, die bei hierarchisch strukturierten Daten zum Einsatz kommen. Die Dimensionalität bleibt dabei erhalten. Beliebt ist beispielsweise das Zusammenfassen von Zeitspannen, also Monaten zu Quartalen oder Jahren.

In Pandas ist ein Umweg von Nöten. Verwenden Sie erneut `groupby` und übergeben Sie eine Liste, deren Werte entweder Namen bereits vorhandener Indizes oder Funktionen sind. Diese Funktionen erhalten als Parameter ein Tupel bestehend aus den indexierten Werten einer Zeile und geben einen Schlüssel zur Gruppierung zurück.

In diesem Fall wird das enthaltene Jahrzehnt auf zwei Positionen reduziert, nämlich alle Werte vor und alle Werte nach (einschließlich) 2010. Durch Funktionen entstandene Indizes besitzen keinen Namen, weshalb dieser manuell gesetzt werden muss.

In [ ]:
rolled_up_cube = cube.groupby(['genre', 'platform', lambda x: x[2] >= 2010]).mean()
rolled_up_cube.index.names = [*rolled_up_cube.index.names[:2], '≥ 2010']

rolled_up_cube.tail(18)

In [ ]:
draw(
    cube,
    rolled_up_cube
)

(Beachten Sie bitte, dass ein Durchschnitt aus Durchschnitten selten das gewünschte Ergebnis hervorbringt und an dieser Stelle zur Veranschaulichung gedacht ist.)

**Drill-Down** ist die Komplementäroperation zu Roll-Up. In diesem Fall wird von Daten zu detaillierten Daten aggregiert. Die Operation ist aber natürlich durch die Granularität der Daten in der Datenbank beschränkt.

## Drill-Across
**Drill-Across** beschreibt eine Operation, bei der zu einem anderen Ergebniswürfel gewechselt wird. Dabei ändert sich jedoch nur die Kennzahl, die Dimensionen bleiben gleich.

## Slice and Dice
**Slice** dient dem Herausschneiden von *Scheiben* aus dem Würfel. Die Dimensionalität wird also verringert, indem z.B. nur die Werte eines einzelnen Genres verwendet werden.

In [ ]:
cube['Rac']

In [ ]:
draw(
    (cube, cube[['Rac']]),
    cube[['Rac']]
)

**Dice** beschreibt das Heraustrennen eines Teil*würfels*. Die Dimensionalität bleibt dabei erhalten. Verwenden Sie die Funktion `loc`, um anhand der Indizes zu trennen.

In [ ]:
cube.loc[['Adv', 'Rac'], ['Com', 'MiS'], 1990:2000]

In [ ]:
draw(
    (cube, cube.loc[['Adv', 'Rac'], ['Com', 'MiS'], 1990:2000]),
    cube.loc[['Adv', 'Rac'], ['Com', 'MiS'], 1990:2000]
)